## <center> COVID-19-Simulation</center>

### Part 0: A SIER cellular automata model:

#### Conceptual Model:
1. Cellular Automata Framework:
2. Geographical Model:
3. Diesase Model:

#### Data Source:
Coronavirus COVID-19 Global Cases by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University (JHU)
- [Blog](https://systems.jhu.edu/research/public-health/ncov/)
- [Dashboard](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)

#### Data used:
[CSSEGISandData](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)
    1. time_series_covid19_confirmed_US.csv
    2. time_series_covid19_confirmed_global.csv
    3. time_series_covid19_deaths_US.csv
    4. time_series_covid19_deaths_global.csv
    5. time_series_covid19_recovered_global.csv
    
    

#### Data Pre-processing, basic statistics and insights:

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def preprocess_US(data_url):
    df = pd.read_csv(data_url)
    last_update_date = df.keys()[-1]
    df.drop(['UID','iso2','iso3','code3','FIPS','Admin2','Lat','Long_'], axis = 1, inplace = True)
    df_cleaned= df.groupby(['Province_State'], as_index = False).sum()
    df_sorted = df_cleaned.sort_values(by=last_update_date,ascending=False).reset_index(drop=True)
    update_total = sum(df_sorted[last_update_date])
    return df_sorted, last_update_date, update_total


In [6]:
### Retrieve data from CSSEGISandData Github Raw content in csv
url_confirmed_US = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url_deaths_US = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'


df_confirmed_US,last_confirmed_US,total_confirmed_US = preprocess_US(url_confirmed_US)
df_deaths_US,last_deaths_US,total_deaths_US = preprocess_US(url_deaths_US)
first_date_US = df_confirmed_US.keys()[1]
print("Data Basic Statistics: \n")
print('From {} till {} the US confirmed cases total is : {} \n'.format(first_date_US,last_confirmed_US,total_confirmed_US))
print('From {} till {} the US deaths cases total is : {} \n'.format(first_date_US,last_deaths_US,total_deaths_US))
mortality_rate = float(total_deaths_US)/float(total_confirmed_US)*100
print('Mortality rate in US is being updated as {:.3f}% \n'.format(mortality_rate))
US_population = df_deaths_US['Population'].sum()
print('COVID-19 is among US population of {:.5f}% and fatal rate is {:.5f}% \n'.format(float(total_confirmed_US)/float(US_population)*100, float(total_deaths_US)/float(US_population)*100))

Data Basic Statistics: 

From 1/22/20 till 4/3/20 the US confirmed cases total is : 275582 

From 1/22/20 till 4/3/20 the US deaths cases total is : 7087 

Mortality rate in US is being updated as 2.572% 

COVID-19 is among US population of 0.08197% and fatal rate is 0.00211% 



In [7]:
df_confirmed_US

,Province_State,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20
0,New York,0,0,0,0,0,0,0,0,0,...,30841,37877,44876,52410,59648,66663,75833,83948,92506,102987
1,New Jersey,0,0,0,0,0,0,0,0,0,...,4402,6876,8825,11124,13386,16636,18696,22255,25590,29895
2,Michigan,0,0,0,0,0,0,0,0,0,...,2296,2845,3634,4650,5488,6498,7615,9315,10791,12744
3,California,0,0,0,0,2,2,2,2,2,...,2998,3899,4657,5095,5852,7138,8210,9399,10773,12004
4,Massachusetts,0,0,0,0,0,0,0,0,0,...,1838,2417,3240,4257,4955,5752,6620,7738,8966,10402
5,Louisiana,0,0,0,0,0,0,0,0,0,...,1795,2304,2744,3315,3540,4025,5237,6424,9159,10297
6,Florida,0,0,0,0,0,0,0,0,0,...,1682,2357,2900,3763,4246,5473,6741,6956,9008,10268
7,Illinois,0,0,1,1,1,1,1,1,1,...,1865,2538,3024,3491,4596,5056,5994,6980,7695,8904
8,Pennsylvania,0,0,0,0,0,0,0,0,0,...,1260,1795,2345,2845,3432,4155,4963,6009,7268,8570
9,Washington,1,1,1,1,1,1,1,1,1,...,2591,3207,3477,4030,4465,4923,5432,5608,6389,6846


In [8]:

df_deaths_US

,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20
0,New York,23628065,0,0,0,0,0,0,0,0,...,285,385,527,728,965,1218,1550,1941,2373,2935
1,New Jersey,8882190,0,0,0,0,0,0,0,0,...,62,81,108,140,161,198,267,355,537,646
2,Michigan,9986857,0,0,0,0,0,0,0,0,...,43,61,92,111,132,184,259,335,417,479
3,Louisiana,4648794,0,0,0,0,0,0,0,0,...,65,83,119,137,151,185,239,273,310,370
4,Washington,7614893,0,0,0,0,0,0,0,0,...,133,150,157,188,198,205,225,234,271,291
5,California,39512223,0,0,0,0,0,0,0,0,...,65,81,94,110,124,146,173,199,238,265
6,Illinois,12671821,0,0,0,0,0,0,0,0,...,19,26,34,47,66,73,99,141,163,210
7,Massachusetts,6892503,0,0,0,0,0,0,0,0,...,15,25,35,44,48,56,89,122,154,192
8,Georgia,10617423,0,0,0,0,0,0,0,0,...,40,48,64,69,80,87,111,139,163,184
9,Florida,21477737,0,0,0,0,0,0,0,0,...,23,29,35,54,56,63,85,87,164,170
